<a href="https://colab.research.google.com/github/databyhuseyn/NLP/blob/main/build_makemore_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
block_size = 3
X, Y = [], []
for w in words[:5]:
  # print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
... ---> a
..a ---> v
.av ---> a
ava ---> .
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [7]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [8]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [72]:
C = torch.randn((27, 2), requires_grad=True)
C

tensor([[-1.4857, -0.7166],
        [ 0.0641, -1.1336],
        [-1.3880,  0.8943],
        [ 1.8821,  0.2960],
        [-0.8075, -0.5792],
        [ 1.5774,  1.5419],
        [ 1.5294,  0.3459],
        [ 0.3239, -0.8447],
        [ 0.5694,  0.2284],
        [ 0.1529,  0.3333],
        [-0.0366,  0.2856],
        [-0.2331,  0.3990],
        [-1.8574,  0.6656],
        [ 0.6570, -0.5125],
        [-0.7576, -1.1268],
        [-2.3140,  0.3900],
        [-0.5094,  0.3089],
        [ 0.1566,  0.7551],
        [ 1.1835, -1.2515],
        [ 0.3526, -1.7500],
        [-1.0412,  0.6442],
        [-0.5829, -1.6310],
        [ 0.4048, -1.6298],
        [ 1.9486, -0.4201],
        [-2.2887, -0.1079],
        [ 1.3564,  0.6291],
        [ 0.7267,  1.9077]], requires_grad=True)

In [10]:
F.one_hot(torch.tensor(5), num_classes = 27).float()

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [12]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [13]:
C[X].shape

torch.Size([32, 3, 2])

In [14]:
X[2]

tensor([ 0,  5, 13])

In [15]:
C[X[2]]

tensor([[-1.9875,  0.4560],
        [-0.0800, -1.0856],
        [-0.6681,  0.1803]])

In [16]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [73]:
W1 = torch.randn((6, 100), requires_grad=True)
b1 = torch.randn(100, requires_grad=True)

In [18]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape

torch.Size([32, 6])

In [19]:
emb[:, 0, :].shape, emb[:, 1, :].shape, emb[:, 2, :].shape

(torch.Size([32, 2]), torch.Size([32, 2]), torch.Size([32, 2]))

In [20]:
# automate previous process
torch.cat(torch.unbind(emb, 1), 1)

tensor([[-1.9875,  0.4560, -1.9875,  0.4560, -1.9875,  0.4560],
        [-1.9875,  0.4560, -1.9875,  0.4560, -0.0800, -1.0856],
        [-1.9875,  0.4560, -0.0800, -1.0856, -0.6681,  0.1803],
        [-0.0800, -1.0856, -0.6681,  0.1803, -0.6681,  0.1803],
        [-0.6681,  0.1803, -0.6681,  0.1803,  1.2728,  1.3444],
        [-1.9875,  0.4560, -1.9875,  0.4560, -1.9875,  0.4560],
        [-1.9875,  0.4560, -1.9875,  0.4560,  0.0802, -0.4106],
        [-1.9875,  0.4560,  0.0802, -0.4106, -0.4062,  2.4792],
        [ 0.0802, -0.4106, -0.4062,  2.4792,  0.5815, -1.0247],
        [-0.4062,  2.4792,  0.5815, -1.0247, -0.1527, -0.8139],
        [ 0.5815, -1.0247, -0.1527, -0.8139,  0.5815, -1.0247],
        [-0.1527, -0.8139,  0.5815, -1.0247,  1.2728,  1.3444],
        [-1.9875,  0.4560, -1.9875,  0.4560, -1.9875,  0.4560],
        [-1.9875,  0.4560, -1.9875,  0.4560,  1.2728,  1.3444],
        [-1.9875,  0.4560,  1.2728,  1.3444, -0.1527, -0.8139],
        [ 1.2728,  1.3444, -0.1527, -0.8

In [21]:
a = torch.arange(18)

In [22]:
a.shape

torch.Size([18])

In [23]:
a.view(3, 3, 2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [24]:
a.storage()

/tmp/ipython-input-214256462.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage()


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [25]:
emb.view(32, 6)       # view is more efficient

tensor([[-1.9875,  0.4560, -1.9875,  0.4560, -1.9875,  0.4560],
        [-1.9875,  0.4560, -1.9875,  0.4560, -0.0800, -1.0856],
        [-1.9875,  0.4560, -0.0800, -1.0856, -0.6681,  0.1803],
        [-0.0800, -1.0856, -0.6681,  0.1803, -0.6681,  0.1803],
        [-0.6681,  0.1803, -0.6681,  0.1803,  1.2728,  1.3444],
        [-1.9875,  0.4560, -1.9875,  0.4560, -1.9875,  0.4560],
        [-1.9875,  0.4560, -1.9875,  0.4560,  0.0802, -0.4106],
        [-1.9875,  0.4560,  0.0802, -0.4106, -0.4062,  2.4792],
        [ 0.0802, -0.4106, -0.4062,  2.4792,  0.5815, -1.0247],
        [-0.4062,  2.4792,  0.5815, -1.0247, -0.1527, -0.8139],
        [ 0.5815, -1.0247, -0.1527, -0.8139,  0.5815, -1.0247],
        [-0.1527, -0.8139,  0.5815, -1.0247,  1.2728,  1.3444],
        [-1.9875,  0.4560, -1.9875,  0.4560, -1.9875,  0.4560],
        [-1.9875,  0.4560, -1.9875,  0.4560,  1.2728,  1.3444],
        [-1.9875,  0.4560,  1.2728,  1.3444, -0.1527, -0.8139],
        [ 1.2728,  1.3444, -0.1527, -0.8

In [26]:
emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1)

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, T

In [27]:
h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)

In [74]:
W2 = torch.randn((100, 27), requires_grad=True)
b2 = torch.randn(27, requires_grad=True)

In [29]:
logits = h @ W2 + b2

In [30]:
logits.shape

torch.Size([32, 27])

In [31]:
counts = logits.exp()   # softmax

In [32]:
prob = counts / counts.sum(1, keepdims = True)    # softmax

In [33]:
prob.shape

torch.Size([32, 27])

In [34]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [60]:
pred_proba = prob[torch.arange(32), Y]
pred_proba

tensor([6.2997e-11, 9.9557e-01, 1.7460e-05, 9.1464e-04, 4.4758e-08, 1.4588e-06,
        7.8709e-13, 4.2092e-12, 4.0597e-10, 4.5261e-07, 1.4269e-01, 1.3363e-15,
        9.6618e-10, 2.0534e-09, 8.3352e-09, 1.7996e-10, 3.7751e-11, 1.4442e-14,
        3.1529e-09, 6.9430e-16, 2.7207e-14, 1.6628e-08, 1.3767e-11, 1.3574e-06,
        4.6600e-03, 5.8108e-16, 3.3420e-05, 2.4421e-14, 5.0820e-09, 1.5989e-09,
        5.3880e-10, 1.5607e-10])